In [1]:
import requests
import pandas as pd

In [2]:
try: # Caching to not DDoS data source
    df = pd.read_csv('stipends.csv')
except:
    df = pd.DataFrame()
    for i in range(0,140):
        response = requests.get('https://www.phdstipends.com/data/'+str(i))
        j = response.json()
        df = pd.concat([df, pd.DataFrame.from_dict(j['data'])])
        # WARNING: They have removed their 'living wage ratio' from their website, presumably due to a data integrity issue.
        # Use Living Wage data from elsewhere for your area.
    df.columns = ['University','Department','Overall Pay', 'Living Wage Ratio','Academic Year','Program Year','Comments','12M Gross Pay','9M Gross Pay', '3M Gross Pay', 'Fees']
    df.to_csv('stipends.csv', index=False)

In [3]:
# Clean comments
df['Comments'] = [str(c).replace('. ', '.<br>').replace(', ', ',<br>').replace(') ', ')<br>') for c in df['Comments']]

In [4]:
# Count number of entries per university
uni_counts = df.groupby('University').size().sort_values(ascending=False)
print(f'Most represented universities:\n{uni_counts.index[:5]}')

Most represented universities:
Index(['University of Wisconsin - Madison (UW)', 'University of Pennsylvania',
       'University of Michigan - Ann Arbor (UM)',
       'University of California - Berkeley',
       'Pennsylvania State University (Penn State)'],
      dtype='object', name='University')


In [5]:
# TODO:
# When overall pay is negative, go by 12M gross pay (because the person entering misunderstood the gross pay/fees field)
# When overall pay is slightly less than 12M gross pay, go by overall pay (because this accounts for dumb little fees)

In [6]:
def format_year(a):
    try:
        return float(a.split('-')[0])
    except:
        return 0
df['Academic Year'] = [format_year(a) for a in df['Academic Year']]
df = df.sort_values(by=['Academic Year'])

In [7]:
def format_pay(p):
    if type(p) is str:
        p = p.replace('$','').replace(',','')
        try:
            return float(p)
        except:
            return p
    else: return p
df['Overall Pay'] = [format_pay(p) for p in df['Overall Pay']]

In [8]:
def format_ratio(r):
    try:
        return float(r)
    except:
        return None
df['Living Wage Ratio'] = [format_ratio(r) for r in df['Living Wage Ratio']]

In [9]:
def format_dept(d):
    # TODO from mappings.json
    return d

In [10]:
def collapse_categories(df):
    # Collapse some categories if we can
    entries = df['Department'].nunique()
    df['Department'] = [str(d).lower().strip() for d in df['Department']]
    print(f"Reduced from {entries} to {df['Department'].nunique()} entries by formatting department strings to lowercase.")
    # TODO: Collapse categories using mappings.json
    df['Aggregate Department'] = [format_dept(d) for d in df['Department']]
    return df

In [11]:
def remove_outliers(df):
    max_pay = 100_000
    min_pay = 10_000
    old_size = df.shape[0]
    df = df[(df['Overall Pay'] < max_pay) & (df['Overall Pay'] > min_pay)]
    print(f'Removed {old_size - df.shape[0]} entries where pay is less than {min_pay} or greater than {max_pay}.')
    old_size = df.shape[0]
    df = df[(df['Academic Year'] > 2010) & (df['Academic Year'] < 2027)]
    print(f'Removed {old_size - df.shape[0]} entries where academic year is outside of 2010-2027.')
    return df

In [12]:
df = collapse_categories(df)
df = remove_outliers(df)

Reduced from 4101 to 3223 entries by formatting department strings to lowercase.
Removed 1213 entries where pay is less than 10000 or greater than 100000.
Removed 98 entries where academic year is outside of 2010-2027.


In [13]:
df.to_csv('cleaned_stipends.csv', index=False)